# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

BFor this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [1]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode


In [128]:
cnx = mysql.connector .connect(
    host = "flatiron.cqkj6tgrp50g.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "Password1"
)

In [129]:
cursor = cnx.cursor()

In [4]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [5]:
## Create new DB 
db_name = 'yelp'

In [6]:
create_database(cursor, db_name)

Failed creating database: 1007 (HY000): Can't create database 'yelp'; database exists


In [7]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [112]:
# Create a table for the Businesses
DB_NAME = 'yelp'

TABLES = {}
TABLES['business'] = (
    "CREATE TABLE business ("
    "  bus_id varchar(100),"
    "  name varchar(100),"
    "  location varchar(100),"
    "  zip_code varchar(5) NOT NULL,"
    "  price varchar(5),"
    "  rating float,"
    "  PRIMARY KEY (bus_id)"
    ") ENGINE=InnoDB")

TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  bus_id varchar(100),"
    "  review_id varchar(100),"
    "  description varchar(100),"
    "  review_count int,"
    "  rating varchar(5) NOT NULL,"
    "  PRIMARY KEY (review_id)"
    ") ENGINE=InnoDB")



In [113]:
TABLES['business']

'CREATE TABLE business (  bus_id varchar(100),  name varchar(100),  location varchar(100),  zip_code varchar(5) NOT NULL,  price varchar(5),  rating float,  PRIMARY KEY (bus_id)) ENGINE=InnoDB'

In [114]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

        
cnx.commit()
cursor.close()
cnx.close()

Creating table business: already exists.
Creating table reviews: OK


## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [11]:
# Write a function to make a call to the yelp API
import requests
import json

In [12]:
# set theurl and endpoint we want to hit
url = 'https://api.yelp.com/v3/businesses/search'

In [21]:
client_id = 'v2Puv78AEi2HDDvyl9K8rg'
api_key = 'OLhy45aVXLmJtClJ75M_1KgoH85lErHLSSSSLTyyeYKb0xTey06_AdUYT3xiEa6hhkHyK-5ttHePmB2wuxIWunAzl5ykGADyj4Ct3tIxkarF14dx8bznH4C4_mHpXXYx'

In [14]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [15]:
# what type of business do you want to search
term = 'Starbucks'

#where do you want to perform this search
location = 'Financial District'

# what is your other parameter you want to search against
status = True

# number of businesses to extract
limit = 50



In [16]:
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status,
                'limit' : limit
            }

In [17]:
url_params

{'term': 'Starbucks',
 'location': 'Financial+District',
 'open_now': True,
 'limit': 50}

In [18]:
def bus_call(url, url_params, api_key):
    response = requests.get(url, headers=headers, params=url_params)
    data_b = response.json()
    return data_b

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [ ]:
biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['location']['zip_code'], business['price'])

In [ ]:
biz_tuple

In [42]:
def parse_buz(l):
    # create a container to hold our parsed data
    parsed_data = []
    
    # loop through our business and parse each individual business
    for business in l:
        try:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['location']['zip_code'], business['price'], business['rating'])
        except:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['location']['zip_code'], None, business['rating'])
    # in the loop, add each individual parsed business to our data container
        parsed_data.append(biz_tuple)
    # return the container with all of the parsed results
    return parsed_data

In [37]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 1500 and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = bus_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_buz(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

TypeError: bus_call() missing 1 required positional argument: 'api_key'

In [ ]:
# Write a function to take your parsed data and insert it into the DB

In [31]:
stmt = """
        INSERT INTO business (bus_id, name, location, zip_code, price) VALUES (%s, %s, %s, %s, %s)
"""

In [ ]:
# Write a script that combines the three functions above into a single process.

In [111]:
cursor.execute("""
                DROP TABLE yelp.reviews""")

ProgrammingError: 1051 (42S02): Unknown table 'yelp.reviews'

In [97]:
cursor = cnx.cursor()

In [130]:
cursor.execute('''USE yelp''')

In [45]:
response = requests.get(url, headers=headers, params=url_params)
data = response.json()
buz_data = parse_buz(data['businesses'])

In [47]:
buz_data[0]

('j6nTfbYavGVsJKxz6bte6w',
 'Starbucks',
 '130 Fulton Street New York, NY 10038',
 '10038',
 '$',
 3.0)

In [40]:
# Combined function
def combined_function():
    cur = 0
    while cur < 1500 and cur < 1000:    
        url_params['offset'] = cur
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()
        buz_data = parse_buz(data['businesses'])
        stmt = """
            INSERT INTO business (bus_id, name, location, zip_code, price, rating) VALUES (%s, %s, %s, %s, %s, %s)
            """
        cursor.executemany(stmt, buz_data)
        cnx.commit()
        cur += 50

In [131]:
combined_function()

IntegrityError: 1062 (23000): Duplicate entry 'j6nTfbYavGVsJKxz6bte6w' for key 'PRIMARY'

In [78]:
len(data)

3

In [84]:
cursor.execute('''SELECT bus_id FROM yelp.business''')
bus_id = cursor.fetchall()

In [172]:
bus_id[0][0]

IndexError: list index out of range

In [ ]:
bus_id[0][0]

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [97]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
client_id = 'v2Puv78AEi2HDDvyl9K8rg'
api_key = '-0Z90lzWTGK-FFDi95t-4bUPkgJnkWfiLGNBL2CqstzNhkgUZU_342EExhX2d-Xy1pwuXKGGX-S7qItCqTEYKnA6eNBZKGjFos_9EfbQXx0IKMtXPwa2LS1M42HpXXYx'

In [23]:
response = requests.get(url, headers=headers, params=url_params)

In [34]:
url_params_r = {
                'locale' : 'en_US'
            }

In [35]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [85]:
bus_id[0][0]

'-2kUyfQrEBxYXfNKfDwAJQ'

In [79]:
    review_url = "https://api.yelp.com/v3/businesses/3EDRLmmA7NEyBAQ86uDzfA/reviews"
    response1 = requests.get(review_url, headers=headers, params=url_params)
    response1.json()

{'reviews': [{'id': '82IiiNiPUxoYLUwAhWn3yQ',
   'url': 'https://www.yelp.com/biz/starbucks-new-york-152?adjust_creative=iEJoh9Ld1tFc56m-OVijoA&hrid=82IiiNiPUxoYLUwAhWn3yQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iEJoh9Ld1tFc56m-OVijoA',
   'text': "I have been to this Starbucks on 16th St and 1st Ave more then a few times and I never had a less then 4 star experience and I bet it's cause the folks who...",
   'rating': 4,
   'time_created': '2019-09-21 11:12:53',
   'user': {'id': 'ILk4dRvuBf6Axfq3qBkoxw',
    'profile_url': 'https://www.yelp.com/user_details?userid=ILk4dRvuBf6Axfq3qBkoxw',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/nO7slGcti7FBYPqRFXWUXw/o.jpg',
    'name': 'Sadri B.'}},
  {'id': 'MD8e0WLOK5zsfmhIwhJMZQ',
   'url': 'https://www.yelp.com/biz/starbucks-new-york-152?adjust_creative=iEJoh9Ld1tFc56m-OVijoA&hrid=MD8e0WLOK5zsfmhIwhJMZQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iEJoh9Ld1tFc56m-OVijoA',


In [108]:
#
def rev_call(bus_id):
    review_url = f"https://api.yelp.com/v3/businesses/{bus_id}/reviews"
    response1 = requests.get(review_url, headers=headers, params=url_params)
    review = response1.json()
    parsed_data = parse_rev(review, bus_id)
    stmt = """
        INSERT INTO reviews (bus_id, review_id, description ,review_count, rating) VALUES (%s, %s, %s, %s, %s)
        """
    cursor.executemany(stmt, parsed_data)
    cnx.commit()

In [116]:
bus_id[0][0]

'-2kUyfQrEBxYXfNKfDwAJQ'

In [117]:
len(bus_id)

391

In [118]:
review_url = f"https://api.yelp.com/v3/businesses/-2kUyfQrEBxYXfNKfDwAJQ/reviews"
response1 = requests.get(review_url, headers=headers, params=url_params)
review = response1.json()

In [119]:
review

{'reviews': [{'id': '_of8URpJZQ3sUcVGwp4X6g',
   'url': 'https://www.yelp.com/biz/starbucks-brooklyn-3?adjust_creative=iEJoh9Ld1tFc56m-OVijoA&hrid=_of8URpJZQ3sUcVGwp4X6g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iEJoh9Ld1tFc56m-OVijoA',
   'text': 'My fave Starbucks with deep bay windows in the back with garden seating even. Such a neighborhood gem.',
   'rating': 5,
   'time_created': '2019-08-17 05:09:13',
   'user': {'id': 'ExhcUTTcj_MBhZdFQT9KYg',
    'profile_url': 'https://www.yelp.com/user_details?userid=ExhcUTTcj_MBhZdFQT9KYg',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/2My2TKnlx_JT9dggGiLBdQ/o.jpg',
    'name': 'Elisa H.'}},
  {'id': 'O9_vXH9_eZ-hicodN9CpwA',
   'url': 'https://www.yelp.com/biz/starbucks-brooklyn-3?adjust_creative=iEJoh9Ld1tFc56m-OVijoA&hrid=O9_vXH9_eZ-hicodN9CpwA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iEJoh9Ld1tFc56m-OVijoA',
   'text': 'Nice atmosphere as usual, great place to have a 

In [126]:
for i in range(0,len(bus_id)):
    try:
        rev_call(bus_id[i][0])
    except:
        continue
        
    
        


In [120]:
# Write a function to parse out the relevant information from the reviews
def parse_rev(business,bus_id):
    # create a container to hold our parsed data
    res_reviews = []
    
    # loop through our business and parse each individual business

    rev_tuple1 = (bus_id, business['reviews'][0]['id'], business['reviews'][0]['text'],business['total'], business['reviews'][0]['rating'])
    rev_tuple2 = (bus_id, business['reviews'][1]['id'], business['reviews'][1]['text'],business['total'], business['reviews'][1]['rating'])
    rev_tuple3 = (bus_id, business['reviews'][2]['id'], business['reviews'][2]['text'],business['total'], business['reviews'][2]['rating'])
    rev_list = [rev_tuple1, rev_tuple2, rev_tuple3]
    res_reviews.extend(rev_list)  
    return res_reviews

In [42]:
review_url = "https://api.yelp.com/v3/businesses/65VJnrsaHS6NZejxnnu23A/reviews"
response1 = requests.get(review_url, headers=headers, params=url_params)
review1 = response1.json()


In [ ]:
response = requests.get(review_api_calls[0], headers=headers, params=url_params_r)


In [ ]:
data1 = response.json()
data['reviews'][0]['id']

In [44]:
def get_rev(review_api_calls):
    rev_list = []
    for i in review_api_calls:
        print(i)
        response = requests.get(i, headers=headers, params=url_params_r)
        
        rev_data = response.json()
        rev = rev_data['reviews']
        print(rev)
        rev_list.append(rev)
    

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews
https://api.yelp.com/v3/businesses/{id}/reviews

In [46]:
# Write a function to parse out the relevant information from the reviews
def parse_rev(l):
    # create a container to hold our parsed data
    parsed_data = []
    
    # loop through our business and parse each individual business
    for business in l:
        rev_tuple = (business['id'], business['name'], business['review_count'], business['rating'])
    # in the loop, add each individual parsed business to our data container
        parsed_data.append(rev_tuple)
    # return the container with all of the parsed results
    return parsed_data

In [ ]:
cursor.executemany(stmt, buz_data)
cnx.commit()

In [48]:
# combine the functions above into a single script 

# Combined function
def combined_function():
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    buz_data = parse_buz(data['businesses'])
    stmt = """
        INSERT INTO business (bus_id, name, location, zip_code, price, rating) VALUES (%s, %s, %s, %s, %s, %f)
        """
    cursor.executemany(stmt, buz_data)
    cnx.commit()
    return 'Done'
    
    

In [127]:
combined_function()

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection


In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50